# Phase 1 - Project 1

## Data Imports

In [ ]:
import csv    #for the cvs and txv files
import sqlite3 as sql   #for the db file
import pandas as pd    #Help visualize the data
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Read All files
Sample the files to see the contents


In [ ]:
df_1 = pd.read_csv("data/tmdb.movies.csv.gz")    #Main use, pull best and worst genre based on rating
df_1.head()

In [ ]:
df_2 = pd.read_csv("data/tn.movie_budgets.csv.gz")
df_2.head()

https://docs.python.org/3/library/codecs.html#standard-encodings - Helped with finding valid encoding 

In [25]:
df_3 = pd.read_csv('data/rt.reviews.tsv.gz',delimiter = '\t', encoding = 'cp437' )
df_3.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [42]:
df_4 = pd.read_csv('data/rt.movie_info.tsv.gz',delimiter = '\t')
df_4.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [ ]:
df_5 = pd.read_csv('data/bom.movie_gross.csv.gz',delimiter = '\t')
df_5.head()

In [ ]:
conn = sql.connect('data/im.db/im.db')

#pd.read_sql('SELECT * FROM  zipfile(data/im.db.zip)', conn)   #https://www.sqlite.org/zipfile.html

In [ ]:
pd.read_sql("""SELECT * FROM movie_ratings""", conn)

## Data Cleaning

### The Movie DB 
Useful keys: Popularity, vote count, vote average

#### Data Exploration

In [ ]:
print(df_1.columns)
df_1['genre_ids']    #What does these numbers mean? - 
df_1['id']    #Each movie has a unique id? 
df_1['original_language']    #Only english movies? - No, there are multiples
df_1['original_title']     #Title of the movie
df_1['popularity']    #Ranking out of ... 100 , Website looks like percentage - based on amount timed watched?
df_1.sort_values('popularity').tail()
#Release data, title are pretty obvious, may need to look in title vs original title
df_1.loc[df_1['vote_average'] == 0]  #Quite a few with not votes and therefore no vote average
#df_1['vote_average'].plot(kind = 'hist')    #- Suject to remove with no votes
#df_1.sort_values(by ='vote_count')

#### Data Filtering

In [ ]:
####   Data Cleaning    ###
print(df_1.info())    #Unknown column
df_1['Unnamed: 0']    #No use, index column
df_1.drop("Unnamed: 0", axis = 1, inplace= True)   #Remove the column
df_1.columns    #Confirm the drop

In [ ]:
###    Check missing values    ###
print (df_1.isna().sum())    #No NaN values
print (df_1.isnull().sum())    #No Null Values


In [ ]:
###    Remove dups    ###
df_1.duplicated().sum()    #There are duplicated movies? Multiple languages?
df_1[df_1.duplicated(keep = False)].sort_values(by = 'id')    #Looks like straight copies - Need to remove
df_1.drop_duplicates(inplace = True)    #Remove dups
df_1.duplicated().sum()   #Check again for dups


#### Data Visualization
Important columns: popularity and genre


In [ ]:
#Sort by popularity to see whats type of movies are the best
df_1.sort_values('popularity', inplace = True,ascending= False)

df_1['genre_ids'] = ((df_1['genre_ids'].map(lambda x: x.strip('[]'))).map(lambda x: x.replace(' ', ''))).map(lambda x: x.split(','))
df_1['genre_ids'][0]
df1_top_movies = df_1[0:100]
df1_worst_movie = df_1[-100:]

In [ ]:
#df1_top_movies
    #for movie in df_list['genre_ids']:
     #   movie = movie.strip('[]')
     #   movie = movie.replace(' ', '')
     #   movie = movie.split(',')
        

##### Graphs

In [ ]:
# Convert the genre id to names for understanding purposes
convert = {'28' : 'Action', '12': 'Adventure', '35': 'Comedy', '99': 'Documentary', 
           '18': 'Drama', '14': 'Fantasy', '27': 'Horror', '878' : 'Sci-Fi', '53' : 'Thriller',
           '16': 'Animation', '80' : 'Crime', '10751' : 'Family','14':'Fantasy',
           "36" : 'History','27': 'Horror', '10402':'Music', '9648':'Mystery',
           '10749':'Romance','10770': 'TV Movie','10752': 'War','37':'Western', '': "NULL" }

# Function count all the genre and group them
def count_genre(df_list):
    genre_count = {}    #https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee

    for genre in df_list['genre_ids']:
        for ids in genre:
            if(genre_count.get(convert[ids])):
                genre_count[convert[ids]] += 1
            else:
                genre_count[convert[ids]] = 1
    return sorted(genre_count.items(), key = lambda x: x[1], reverse=True)
   
df1_top_genre = count_genre(df1_top_movies)
df1_worst_genre = count_genre(df1_worst_movie)
x, y = zip(*df1_top_genre)
x2,y2 = zip(*df1_worst_genre)


fig, ax = plt.subplots(1,2, figsize = (20,10))
colors1 = ['red' if genre in x2[0:5] else 'green' for genre in x]
colors2 = ['cyan' if genre in x[0:5] else 'black' for genre in x2]

ax[0].barh(x,y, color = colors1);
ax[1].barh(x2,y2, color = colors2);

ax[1].set_xlabel("Frequency");
ax[0].set_xlabel("Frequency");

ax[0].set_ylabel("Genre");
ax[1].set_ylabel("Genre");

ax[1].set_title("Worst Genre - Worst 100 Movies");
ax[0].set_title("Best Genre - Top 100 Movies");

ax[0].legend(["Good Tags"]);
ax[1].legend(["Bad Tags"]);

### The Number

#### Data Exploration

In [ ]:
df_2.duplicated().sum()    #No dups?
df_2.isna().sum()    #No NaN
df_2.isnull().sum()    #No Null
df_2.info()    #All usable except release data right now

#### Data Filtering

In [ ]:
###    Change the columns to integer for comparing and statistical methods    ###

df_2['production_budget'] = df_2['production_budget'].map(lambda x: x.strip('$'))    #Take out the $
df_2['production_budget'] = df_2['production_budget'].map(lambda x: x.replace(',',''))    #Remove the , 
df_2['production_budget'] = df_2['production_budget'].astype('int64')    #Convert to int64

df_2['domestic_gross'] = df_2['domestic_gross'].map(lambda x: x.strip('$'))    #Take out the $
df_2['domestic_gross'] = df_2['domestic_gross'].map(lambda x: x.replace(',',''))    #Remove the , 
df_2['domestic_gross'] = df_2['domestic_gross'].astype('int64')

df_2['worldwide_gross'] = df_2['worldwide_gross'].map(lambda x: x.strip('$'))   #Take out the $
df_2['worldwide_gross'] = df_2['worldwide_gross'].map(lambda x: x.replace(',',''))    #Remove the , 
df_2['worldwide_gross'] = df_2['worldwide_gross'].astype('int64')
df_2.info()


In [ ]:
df_2
df_2['prod_dom_ratio'] = df_2['domestic_gross'] / df_2['production_budget']    #Ratio on the more proftitable movie
df_2.sort_values(['prod_dom_ratio'], ascending= False, inplace= True)
df_2.head(10)     #One outlier...I see


print((df_2['prod_dom_ratio'] > 1).sum())    #Half the movie broke even - Categorical?
print((df_2['prod_dom_ratio'] > 2).sum())    #1/4 of the movie doubled <- Goal
print((df_2['prod_dom_ratio'] > 3).sum())    #Best case for most movies 
df2_most_prof_3 = df_2['movie'].loc[df_2['prod_dom_ratio'] > 3]  #Are the movies from the first data in this list?
df_2['movie'].loc[df_2['movie'] == 'Avengers: Infinity War']    #In the list


##### Data Comparison - Ratio of 3 and up

In [ ]:
# Titles from the first dataset
df1_top_list = list(df1_top_movies['title'])
df1_top_list


In [ ]:

df1_n_df2_3 = []
for name in df2_most_prof_3:
    for top in df1_top_list:
        if(name == top):
            df1_n_df2_3.append(name)
df_1.loc[df_1['title'].isin(df1_n_df2_3)]
over_3 = count_genre(df_1.loc[df_1['title'].isin(df1_n_df2_3)])

##### Data Comparison - Ratio of 2 and up

In [ ]:
df2_most_prof_2 = df_2['movie'].loc[df_2['prod_dom_ratio'] > 2]
df2_most_prof
df1_n_df2_2 = []
for name in df2_most_prof_2:
    for top in df1_top_list:
        if(name == top):
            df1_n_df2_2.append(name)
df_1.loc[df_1['title'].isin(df1_n_df2_2)]
over_2 = count_genre(df_1.loc[df_1['title'].isin(df1_n_df2_2)])
print(df1_n_df2_2)

In [ ]:
df2_most_prof = df_2['movie'].loc[df_2['prod_dom_ratio'] > 1]
df2_most_prof
df1_n_df2 = []
for name in df2_most_prof:
    for top in df1_top_list:
        if(name == top):
            df1_n_df2.append(name)
df_1.loc[df_1['title'].isin(df1_n_df2)]
over_1 = count_genre(df_1.loc[df_1['title'].isin(df1_n_df2)])
print(df1_n_df2)


#### Data Visualization

In [ ]:
fig, ax = plt.subplots(3,1, figsize = (10, 15))
x, y = zip(*over_1)
x2, y2 = zip(*over_2)
x3, y3 = zip(*over_3)
ax[0].barh(x,y)
ax[1].barh(x2,y2)
ax[2].barh(x3,y3)

### Rotten Tomatoes

Id is probably Movie ID 
Review are the written review of the critic
Rating is out of 5
Freshness appears to be 3>Fresh and lower is rotten
Top critics are acclaimed critics on rottom tomatoes
Publisher and Release data have no use at the moment


In [77]:
df_3.head()
#df_3.info()
#df_3.isna().sum()
df_3.loc[df_3['review'].isna()]
df3_top_critic = df_3.loc[df_3['top_critic'] == 1]
print(len(df3_top_critic['critic'].unique()))    #856 unique critics
print(len(df3_top_critic))    #13096 review from top critics


856
13096


In [84]:
df3_count_review = {}
df3_top_critic.head()


,id,review,rating,fresh,critic,top_critic,publisher,date
24,3,"The rapid dialogue is dry and mannered, like a...",2.5/4,rotten,Joe Williams,1,St. Louis Post-Dispatch,"September 6, 2012"
47,3,It feels like each and every moment bursts for...,2/4,rotten,Stephanie Merry,1,Washington Post,"August 24, 2012"
50,3,"There's not really a movie there, nothing that...",2/4,rotten,Mick LaSalle,1,San Francisco Chronicle,"August 23, 2012"
54,3,The film is all too faithful to its un-cinemat...,1.5/4,rotten,Colin Covert,1,Minneapolis Star Tribune,"August 23, 2012"
55,3,"Cosmopolis,"" because of its allegiance to the ...",2/4,rotten,Moira MacDonald,1,Seattle Times,"August 23, 2012"


In [86]:
df_4.head()
df_4.sort_values('box_office')    #Not alot of info like the other data set
df_4.isna().sum()    #1560 entires, 1220 are NaN. Data set not usable for current project.
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [87]:
df_5

,"title,studio,domestic_gross,foreign_gross,year"
0,"Toy Story 3,BV,415000000.0,652000000,2010"
1,"Alice in Wonderland (2010),BV,334200000.0,6913..."
2,"Harry Potter and the Deathly Hallows Part 1,WB..."
3,"Inception,WB,292600000.0,535700000,2010"
4,"Shrek Forever After,P/DW,238700000.0,513900000..."
...,...
3382,"The Quake,Magn.,6200.0,,2018"
3383,"Edward II (2018 re-release),FM,4800.0,,2018"
3384,"El Pacto,Sony,2500.0,,2018"
3385,"The Swan,Synergetic,2400.0,,2018"
